In [3]:
!pip install gym
!pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk
import gym, gym_walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-3l9zspxa/gym-walk_bfd0a2c83586444c8026727c0dcf9711
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-3l9zspxa/gym-walk_bfd0a2c83586444c8026727c0dcf9711
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=4053 sha256=c7a02bcd2204a38eaebc01301e3ef78df1ad78089ff812db25e5ab44213385b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mpf8qsms/wheels/24/fe/c4/0cbc7511d29265bad7e28a09311db3f87f0cafba74af54d530
Successfully built gym-walk


In [10]:
import matplotlib.pyplot as plt
from IPython import display
import time

### Bandit Walk

In [8]:
#BW
P = {
    0:{
        0: [(1.0,0,0.0, True)],
        1: [(1,0,0,0.0, True)]
    },
    1:{
         0: [(1.0,0,0.0, True)],
         1: [(1.0,2,1.0, True)]
    },
    2:{
        0: [(1.0,2,0.0, True)],
        1: [(1,0,2,0.0, True)]
    }
}
P1 = gym.make('BanditWalk-v0').env.P

In [15]:
import numpy as np

class TDAgent:
    def __init__(self, num_states, alpha=0.1, gamma=0.99, epsilon=0.1):
        self.num_states = num_states
        self.alpha, self.gamma, self.epsilon = alpha, gamma, epsilon
        self.values = np.zeros(num_states)

    def choose_action(self, state):
        return np.random.choice([0, 1]) if np.random.rand() < self.epsilon else np.argmax(self.values[state])

    def update_value(self, state, reward, next_state):
        td_target = reward + self.gamma * self.values[next_state]
        self.values[state] += self.alpha * (td_target - self.values[state])

# Environment transition probabilities dictionary
P = {
    0: {
        0: [(1.0, 0, 0.0, True)],
        1: [(1, 0, 0, 0.0, True)]
    },
    1: {
        0: [(1.0, 0, 0.0, True)],
        1: [(1.0, 2, 1.0, True)]
    },
    2: {
        0: [(1.0, 2, 0.0, True)],
        1: [(1, 0, 2, 0.0, True)]
    }
}

# Parameters
num_episodes, num_states = 1000, len(P)

# Initialize
agent = TDAgent(num_states)

# Training loop
for _ in range(num_episodes):
    state, done = 1, False  # Start from state 1
    while not done:
        action = agent.choose_action(state)
        prob, next_state, reward, done = P[state][action][0]  # Get next state and reward from transition probabilities
        agent.update_value(state, reward, next_state)
        state = next_state

# After training
print("Learned values:")
print(agent.values)

Learned values:
[0.         0.02580032 0.        ]


### Bandit Slippery Walk

In [9]:
#BSW
P = {
    0:{
        0: [(1.0,0,0.0, True)],
        1: [(1,0,0,0.0, True)]
    },
    1:{
         0: [(0.8,0,0.0, True), (0.2,2,1.0, True)],
         1: [(0.8,2,1.0, True), (0.2,0,0.0, True)]
    },
    2:{
        0: [(1.0,2,0.0, True)],
        1: [(1,0,2,0.0, True)]
    }
}
P2 = gym.make('BanditSlipperyWalk-v0').env.P

In [16]:
import numpy as np

class TDAgent:
    def __init__(self, num_states, alpha=0.1, gamma=0.99, epsilon=0.1):
        self.num_states = num_states
        self.alpha, self.gamma, self.epsilon = alpha, gamma, epsilon
        self.values = np.zeros(num_states)

    def choose_action(self, state):
        return np.random.choice([0, 1]) if np.random.rand() < self.epsilon else np.argmax(self.values[state])

    def update_value(self, state, reward, next_state):
        td_target = reward + self.gamma * self.values[next_state]
        self.values[state] += self.alpha * (td_target - self.values[state])

# Bandit Slippery walk environment transition probabilities dictionary
P = {
    0: {
        0: [(1.0, 0, 0.0, True)],
        1: [(1.0, 0, 0.0, True)]
    },
    1: {
        0: [(0.8, 0, 0.0, True), (0.2, 2, 1.0, True)],
        1: [(0.8, 2, 1.0, True), (0.2, 0, 0.0, True)]
    },
    2: {
        0: [(1.0, 2, 0.0, True)],
        1: [(1, 0, 2, 0.0, True)]
    }
}

# Parameters
num_episodes, num_states = 1000, len(P)

# Initialize
agent = TDAgent(num_states)

# Training loop
for _ in range(num_episodes):
    state, done = 1, False  # Start from state 1
    while not done:
        action = agent.choose_action(state)
        transitions = P[state][action]
        prob, next_state, reward, done = transitions[np.random.choice(len(transitions))]
        agent.update_value(state, reward, next_state)
        state = next_state

# After training
print("Learned values:")
print(agent.values)

Learned values:
[0.        0.3420852 0.       ]


### Frozen Lake

In [29]:
import numpy as np

class TDAgent:
    def __init__(self, num_states, alpha=0.1, gamma=0.99, epsilon=0.1):
        self.num_states = num_states
        self.alpha, self.gamma, self.epsilon = alpha, gamma, epsilon
        self.values = np.random.rand(num_states)  # Initialize with random values

    def choose_action(self, state):
        return np.random.choice([0, 1, 2, 3]) if np.random.rand() < self.epsilon else np.argmax(self.values[state])

    def update_value(self, state, reward, next_state):
        td_target = reward + self.gamma * self.values[next_state]
        self.values[state] += self.alpha * (td_target - self.values[state])

# Frozen Lake environment transition probabilities dictionary
P = {
    0:{
        0: [(0.33,0,0.0,False),(0.33,0,0.0,False),(0.33,4,0.0,False)],
        1: [(0.33,4,0.0,False),(0.33,0,0.0,False),(0.33,1,0.0,False)],
        2: [(0.33,1,0.0,False),(0.33,4,0.0,False),(0.33,0,0.0,False)],
        3: [(0.33,0,0.0,False),(0.33,0,0.0,False),(0.33,1,0.0,False)]
    },
    1:{
        0: [(0.33,0,0.0,False),(0.33,1,0.0,False),(0.33,5,0.0,True)],
        1: [(0.33,0,0.0,False),(0.33,5,0.0,True),(0.33,2,0.0,False)],
        2: [(0.33,5,0.0,True),(0.33,2,0.0,False),(0.33,1,0.0,False)],
        3: [(0.33,0,0.0,False),(0.33,1,0.0,False),(0.33,2,0.0,False)]
    },
    2:{
        0: [(0.33,1,0.0,False),(0.33,2,0.0,False),(0.33,6,0.0,False)],
        1: [(0.33,1,0.0,False),(0.33,3,0.0,False),(0.33,6,0.0,False)],
        2: [(0.33,2,0.0, False),(0.33,3,0.0, False),(0.33,6,0.0,False)],
        3: [(0.33,1,0.0, False),(0.33,2,0.0,False),(0.33,3,0.0,False)]
    },
    3:{
        0: [(0.33,2,0.0,False),(0.33,3,0.0,False),(0.33,7,0.0,True)],
        1: [(0.33,7,0.0,True),(0.33,2,0.0,False),(0.33,3,0.0,False)],
        2: [(0.33,3,0.0,False),(0.33,3,0.0,False),(0.33,7,0.0,True)],
        3: [(0.33,2,0.0,False),(0.33,3,0.0,False),(0.33,3,0.0,False)]
    },
    4:{
        0: [(0.33,4,0.0,False),(0.33,0,0.0,False),(0.33,8,0.0,False)],
        1: [(0.33,4,0.0,False),(0.33,8,0.0,False),(0.33,5,0.0,True)],
        2: [(0.33,0,0.0, False),(0.33,5,0.0,True),(0.33,8,0.0,False)],
        3: [(0.33,0,0.0, False),(0.33,4,0.0,False),(0.33,5,0.0,True)]
    },
    5:{
        0: [(1,5,0.0,True)],
        1: [(1,5,0.0,True)],
        2: [(1,5,0.0,True)],
        3: [(1,5,0.0,True)]
    },
    6:{
        0: [(0.33,5,0.0,True),(0.33,2,0.0,False),(0.33,10,0.0,False)],
        1: [(0.33,10,0.0,False),(0.33,5,0.0,True),(0.33,7,0.0,True)],
        2: [(0.33,10,0.0,False),(0.33,2,0.0,False),(0.33,7,0.0,True)],
        3: [(0.33,2,0.0, False),(0.33,5,0.0,True),(0.33,7,0.0,True)]
    },
    7:{
        0: [(1,7,0.0,True)],
        1: [(1,7,0.0,True)],
        2: [(1,7,0.0,True)],
        3: [(1,7,0.0,True)]
    },
    8:{
        0: [(0.33,4,0.0,False),(0.33,8,0.0,False),(0.33,12,0.0,True)],
        1: [(0.33,8,0.0,False),(0.33,12,0.0,True),(0.33,9,0.0,False)],
        2: [(0.33,4,0.0,False),(0.33,9,0.0,False),(0.33,12,0.0,True)],
        3: [(0.33,8,0.0,False),(0.33,4,0.0,False),(0.33,9,0.0,False)]
    },
    9:{
        0: [(0.33,5,0.0,True),(0.33,8,0.0,False),(0.33,13,0.0,False)],
        1: [(0.33,8,0.0,False),(0.33,13,0.0,False),(0.33,10,0.0,False)],
        2: [(0.33,5,0.0,True),(0.33,10,0.0,False),(0.33,13,0.0,False)],
        3: [(0.33,8,0.0,False),(0.33,5,0.0,True),(0.33,10,0.0,False)]
    },
    10:{
        0: [(0.33,6,0.0,False),(0.33,9,0.0,False),(0.33,14,0.0,False)],
        1: [(0.33,9,0.0,False),(0.33,14,0.0,False),(0.33,11,0.0,True)],
        2: [(0.33,14,0.0,False),(0.33,11,0.0,True),(0.33,6,0.0,False)],
        3: [(0.33,6,0.0,False),(0.33,9,0.0,False),(0.33,11,0.0,True)]
    },
    11:{
        0: [(1,11,0.0,True)],
        1: [(1,11,0.0,True)],
        2: [(1,11,0.0,True)],
        3: [(1,11,0.0,True)]
    },
    12:{
        0: [(1,12,0.0,True)],
        1: [(1,12,0.0,True)],
        2: [(1,12,0.0,True)],
        3: [(1,12,0.0,True)]
    },
    13:{
        0: [(0.33,12,0.0,True),(0.33,13,0.0,False),(0.33,9,0.0,False)],
        1: [(0.33,14,0.0,False),(0.33,13,0.0,False),(0.33,12,0.0,True)],
        2: [(0.33,13,0.0,False),(0.33,14,0.0,False),(0.33,9,0.0,False)],
        3: [(0.33,12,0.0,True),(0.33,9,0.0,False),(0.33,14,0.0,False)]
    },
    14:{
        0: [(0.33,10,0.0,False),(0.33,13,0.0,False),(0.33,14,0.0,False)],
        1: [(0.33,13,0.0,False),(0.33,14,0.0,False),(0.33,15,1.0,True)],
        2: [(0.33,14,0.0,False),(0.33,15,1.0,True),(0.33,10,0.0,False)],
        3: [(0.33,15,1.0,True),(0.33,10,0.0,False),(0.33,13,0.0,False)]
    },
    15:{
        0: [(1.0, 15, 0.0, True)],
        1: [(1.0, 15, 0.0, True)],
        2: [(1.0, 15, 0.0, True)],
        3: [(1.0, 15, 0.0, True)]
    }
}
# Parameters
num_episodes, num_states = 20, len(P)

# Initialize
agent = TDAgent(num_states)

# Training loop
for _ in range(num_episodes):
    state, done = 0, False  # Start from state 0
    while not done:
        action = agent.choose_action(state)
        transitions = P[state][action]
        prob, next_state, reward, done = transitions[np.random.choice(len(transitions))]
        agent.update_value(state, reward, next_state)
        state = next_state

# After training
print("Learned values (4x4 grid form):")
grid_values = agent.values.reshape((4, 4))
print(grid_values)


Learned values (4x4 grid form):
[[0.32197521 0.44965388 0.51307932 0.51733467]
 [0.31062721 0.70459221 0.59888972 0.31396492]
 [0.30226044 0.14913084 0.12387068 0.12782308]
 [0.32834289 0.4562168  0.75125644 0.23785396]]
